In [1]:
import pandas as pd
import numpy as np

### Reading in data of all australian crimes by local government area, suburb and offences

### renaming columns

In [9]:
aus_crime_df=pd.read_csv('../003_Extract_Transform_Crime_Data/01_Outputs/suburb_vs_crime.csv')

### Making all string values lower case

In [11]:
aus_crime_df['lga_name'] = aus_crime_df['lga_name'].str.lower()
aus_crime_df['suburb'] = aus_crime_df['suburb'].str.lower()
aus_crime_df

,lga_name,postcode,suburb,crime_id,crime_name,average_incident
0,banyule,3079,ivanhoe,A20,Assault and related offences,43.600000
1,banyule,3079,ivanhoe,A50,Robbery,3.000000
2,banyule,3079,ivanhoe,A70,"Stalking, harassment and threatening behaviour",8.100000
3,banyule,3079,ivanhoe,A80,Dangerous and negligent acts endangering people,3.333333
4,banyule,3079,ivanhoe,B10,Arson,3.400000
...,...,...,...,...,...,...
7512,yarra ranges,3796,mount evelyn,D30,Public nuisance offences,1.833333
7513,yarra ranges,3796,mount evelyn,E10,Justice procedures,4.250000
7514,yarra ranges,3796,mount evelyn,E20,Breaches of orders,21.800000
7515,yarra ranges,3796,mount evelyn,F90,Miscellaneous offences,4.333333


### reading in suburbs of greater melbourne csv 

In [13]:
melb_suburbs_df = pd.read_csv('01_Outputs/melbourne_suburbs.csv')
melb_suburbs_df['suburb'] = melb_suburbs_df['suburb'].str.lower()
melb_suburbs_df

,suburb
0,abbotsford
1,aberfeldie
2,airport west
3,albanvale
4,albert park
...,...
316,williamstown
317,williamstown north
318,windsor
319,yallambie


### merging aus_crime_df_group with melb_suburbs_df for only melbourne suburbs

In [14]:
merged_melb_subs = aus_crime_df.merge(melb_suburbs_df, on = 'suburb', how = 'inner')
merged_melb_subs

,lga_name,postcode,suburb,crime_id,crime_name,average_incident
0,banyule,3079,ivanhoe,A20,Assault and related offences,43.600000
1,banyule,3079,ivanhoe,A50,Robbery,3.000000
2,banyule,3079,ivanhoe,A70,"Stalking, harassment and threatening behaviour",8.100000
3,banyule,3079,ivanhoe,A80,Dangerous and negligent acts endangering people,3.333333
4,banyule,3079,ivanhoe,B10,Arson,3.400000
...,...,...,...,...,...,...
7512,yarra ranges,3796,mount evelyn,D30,Public nuisance offences,1.833333
7513,yarra ranges,3796,mount evelyn,E10,Justice procedures,4.250000
7514,yarra ranges,3796,mount evelyn,E20,Breaches of orders,21.800000
7515,yarra ranges,3796,mount evelyn,F90,Miscellaneous offences,4.333333


### grouping by lga suburb and postcode for sum of total incidnets 

In [57]:
sub_lga_post = merged_melb_subs[['lga_name','suburb','postcode']]

In [58]:
sub_lga_post

,lga_name,suburb,postcode
0,banyule,ivanhoe,3079
1,banyule,ivanhoe,3079
2,banyule,ivanhoe,3079
3,banyule,ivanhoe,3079
4,banyule,ivanhoe,3079
...,...,...,...
7512,yarra ranges,mount evelyn,3796
7513,yarra ranges,mount evelyn,3796
7514,yarra ranges,mount evelyn,3796
7515,yarra ranges,mount evelyn,3796


# id and lga table

### creating list of numbers to give ids for local government areas

In [59]:
len(list(sub_lga_post['lga_name'].unique()))


32

In [60]:
numbers = [x for x in range(1,33)]
lga_name = list(sub_lga_post['lga_name'].unique())

In [61]:
local_govt_areas = pd.DataFrame({
    'lga_id':numbers,
    'lga_name':lga_name})


In [62]:
local_govt_areas

,lga_id,lga_name
0,1,banyule
1,2,northern grampians
2,3,darebin
3,4,whittlesea
4,5,nillumbik
5,6,bayside
6,7,glen eira
7,8,kingston
8,9,boroondara
9,10,whitehorse


In [32]:
local_govt_areas.to_csv('../005_Load_SQL/01_Inputs/local_govt_areas.csv', index = False)

# suburb postcode table

In [63]:
x = pd.DataFrame(sub_lga_post.groupby(['suburb','postcode'])['postcode'].count())

In [64]:
x = x.drop(columns='postcode')
x = x.reset_index()


In [65]:
sub_post = x.drop_duplicates(subset = 'suburb')

In [66]:
sub_post

,suburb,postcode
0,abbotsford,3067
1,aberfeldie,3040
2,airport west,3042
3,albanvale,3021
4,albert park,3206
...,...,...
332,williamstown,3016
333,williamstown north,3016
334,windsor,3181
335,yallambie,3085



# linking table

In [70]:
dropped_df = sub_lga_post.drop_duplicates(subset='suburb')
= dropped_df.merge(local_govt_areas, on='lga_name', how='inner')

,lga_name,suburb,postcode,lga_id
0,banyule,ivanhoe,3079,1
1,banyule,ivanhoe east,3079,1
2,banyule,bellfield,3081,1
3,banyule,heidelberg heights,3081,1
4,banyule,heidelberg west,3081,1
...,...,...,...,...
308,yarra ranges,ferny creek,3786,20
309,yarra ranges,sherbrooke,3789,20
310,yarra ranges,kallista,3791,20
311,yarra ranges,the patch,3792,20


In [30]:
linking_table.to_csv('01_Outputs/lga_suburbs.csv', index = False)

In [31]:
linking_table.to_csv('../005_Load_SQL/01_Inputs/lga_suburbs.csv', index = False)